In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/

In [3]:
import pandas as pd
#import pandas_profiling
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import optuna.integration.lightgbm as lgb
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from xgboost.sklearn import XGBClassifier


print("OK\n")

OK



In [4]:
app_train =  pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
app_test =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
# 他社債務
hc_br_train = pd.read_csv('./home-credit-default-risk/exports/hc_br_train.csv')
hc_br_test = pd.read_csv('./home-credit-default-risk/exports/hc_br_test.csv')
# hc Cash loans
hc_only_cash_train = pd.read_csv('./home-credit-default-risk/exports/hc_only_cash_train.csv')
hc_only_cash_test = pd.read_csv('./home-credit-default-risk/exports/hc_only_cash_test.csv')
# hc Revolbing loans
hc_only_revo_train = pd.read_csv('./home-credit-default-risk/exports/hc_only_revo_train.csv')
hc_only_revo_test = pd.read_csv('./home-credit-default-risk/exports/hc_only_revo_test.csv')
## hc Consumer loans
hc_only_consumer_train = pd.read_csv('./home-credit-default-risk/exports/hc_only_consumer_train.csv')
hc_only_consumer_test = pd.read_csv('./home-credit-default-risk/exports/hc_only_consumer_test.csv')

In [5]:
columns = ['TARGET']
hc_br_train.drop(columns, axis=1, inplace=True)
hc_only_cash_train.drop(columns, axis=1, inplace=True)
hc_only_revo_train.drop(columns, axis=1, inplace=True)
hc_only_consumer_train.drop(columns, axis=1, inplace=True)

In [6]:
# trainへマージ
app_train = pd.merge(app_train, hc_br_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_br_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [7]:
# trainへマージ
app_train = pd.merge(app_train, hc_only_cash_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_only_cash_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [8]:
# trainへマージ
app_train = pd.merge(app_train, hc_only_revo_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_only_revo_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [9]:
# trainへマージ
app_train = pd.merge(app_train, hc_only_consumer_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_only_consumer_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [10]:
#num_cols = ['max_credit_day_overdue', 'max_days_credit_enddate', 'min_days_enddate_fact', 'max_amt_credit_max_overdue', 'cnt_credit_prolong_in_6month', 'sum_amt_credit_sum', 'sum_amt_credit_sum_debt', 'sum_amt_credit_sum_limit', 'sum_amt_credit_sum_overdue', 'avg_amt_annuity']

data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

# 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(X_train[num_cols])

# 変換後のデータで各列を置換
#X_train[num_cols] = pt.transform(X_train[num_cols])
#X_test[num_cols] = pt.transform(X_test[num_cols])

X_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,...,consumer_amt_application,consumer_amt_credit,consumer_amt_goods_price,consumer_weekday_appr_process_start,consumer_hour_appr_process_start,consumer_name_client_type,consumer_name_goods_category,consumer_name_seller_industry,consumer_days_last_due,consumer_installment_payment_ratio_1000_mean_mean
0,100002,0,1,0,0,202500.0,406597.5,24700.5,351000.0,0,...,179055.0,179055.0,179055.0,0.0,9.0,0.0,0.0,0.0,-25.0,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,0,0,135000.0,312682.5,29686.5,297000.0,0,...,334917.0,267930.0,334917.0,1.0,15.0,1.0,1.0,1.0,365243.0,0.0
4,100007,0,1,0,0,121500.0,513000.0,21865.5,513000.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
import xgboost as xgb
from sklearn.metrics import log_loss

y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = columns

params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    ,'num_round': 10000
    ,'early_stopping_round': 50
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 
    
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

[22:03:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "early_stopping_round", "num_round", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.72933	eval-auc:0.71639
[1]	train-auc:0.74266	eval-auc:0.72646
[2]	train-auc:0.75264	eval-auc:0.73268
[3]	train-auc:0.75643	eval-auc:0.73405
[4]	train-auc:0.75992	eval-auc:0.73694
[5]	train-auc:0.76312	eval-auc:0.73839
[6]	train-auc:0.76455	eval-auc:0.73865
[7]	train-auc:0.76652	eval-auc:0.73994
[8]	train-auc:0.77361	eval-auc:0.74380
[9]	train-auc:0.77631	eval-auc:0.74531
[10]	train-auc:0.77794	eval-auc:0.74525
[11]	train-auc:0.77934	eval-auc:0.74596
[12]	train-auc:0.78088	eval-auc:0.74695
[13]	t

[8]	train-auc:0.77281	eval-auc:0.74167
[9]	train-auc:0.77550	eval-auc:0.74367
[10]	train-auc:0.77695	eval-auc:0.74390
[11]	train-auc:0.77908	eval-auc:0.74444
[12]	train-auc:0.78060	eval-auc:0.74488
[13]	train-auc:0.78122	eval-auc:0.74554
[14]	train-auc:0.78218	eval-auc:0.74572
[15]	train-auc:0.78338	eval-auc:0.74608
[16]	train-auc:0.78545	eval-auc:0.74742
[17]	train-auc:0.78743	eval-auc:0.74792
[18]	train-auc:0.78888	eval-auc:0.74879
[19]	train-auc:0.79127	eval-auc:0.74996
[20]	train-auc:0.79277	eval-auc:0.75007
[21]	train-auc:0.79416	eval-auc:0.75025
[22]	train-auc:0.79595	eval-auc:0.75036
[23]	train-auc:0.79718	eval-auc:0.75074
[24]	train-auc:0.79857	eval-auc:0.75126
[25]	train-auc:0.80000	eval-auc:0.75175
[26]	train-auc:0.80209	eval-auc:0.75243
[27]	train-auc:0.80326	eval-auc:0.75298
[28]	train-auc:0.80452	eval-auc:0.75310
[29]	train-auc:0.80594	eval-auc:0.75333
[30]	train-auc:0.80712	eval-auc:0.75388
[31]	train-auc:0.80830	eval-auc:0.75441
[32]	train-auc:0.80986	eval-auc:0.75448
[3

In [12]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]
submit_all.isnull().sum()

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [13]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [14]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [15]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [16]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [17]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [18]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
0,0.046665,0.089254,0.050364,0.074546,0.050917
1,0.072501,0.094397,0.074386,0.092039,0.064493
2,0.040284,0.038702,0.042843,0.023602,0.057926
3,0.087806,0.048217,0.045236,0.063764,0.037870
4,0.131192,0.127920,0.108539,0.108557,0.147926


In [19]:
submit_all.corr()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
submit_all_00,1.000000,0.905150,0.907997,0.908306,0.901900
submit_all_01,0.905150,1.000000,0.907356,0.905823,0.896856
submit_all_02,0.907997,0.907356,1.000000,0.910213,0.900187
submit_all_03,0.908306,0.905823,0.910213,1.000000,0.900204
submit_all_04,0.901900,0.896856,0.900187,0.900204,1.000000


In [20]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.062349
1,100005,0.079563
2,100013,0.040671
3,100028,0.056579
4,100038,0.124827


In [21]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/submit_all.csv', index=False)
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.012470
1,100005,0.015913
2,100013,0.008134
3,100028,0.011316
4,100038,0.024965


In [22]:
submit_all.count()

SK_ID_CURR    48744
TARGET        48744
dtype: int64